# Sumarizador Automático de Texto usando Embeddings 
A proposta deste trabalho é desenvolver uma Prova de Conceito no intuito de demonstrar um sumarizador automático de texto capaz de gerar resumos extrativos de documentos usando embeddings para capturar a semântica das frases. Para os testes e avaliação, será utilizado o dataset CNN/DailyMail; ele contém artigos de notícias e seus resumos, e é amplamente utilizado para tarefas de sumarização. 

O pipeline de execução do sumarizador será composto, basicamente, pelas seguintes etapas: 
1. Transformar cada frase do texto em um vetor (embedding); 
2. Clusterizar esses embeddings; 
3. Ranquear as frases mais importantes de cada cluster; 
4. Compor o resumo extrativo a partir das frases obtidas.

Neste projeto serão usados o banco de dados vetorial ``Milvus Standalone`` e o modelo de embedding ``text-embedding-3-small`` da OpenAI:.


```Observações:```

- Para rodar o notebook, é necessário ter um servidor do Milvus rodando na máquina local. Para isso, é necessário ter o Docker instalado e em execução, e rodar os seguintes comandos no terminal:
    1. Baixar os arquivos do Milvus:
        ```sh
        wget https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh
        ```
    2. Iniciar o Milvus
        ```sh
        bash standalone_embed.sh start
        ```
 - Para parar o Milvus após o uso, usar: 
    ```sh
    bash standalone_embed.sh stop
    ```
- Para deletar os dados do Milvus:
    ```sh
    bash standalone_embed.sh delete
    ```
 - No caso de sistema operacional Windows, é necessário usar o WSL2.

---

## Instalação de Dependências

In [1]:
# Uncomment
# ! pip install pymilvus==2.4.1 scikit-learn nltk datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.8/308.8 kB 5.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.5.0
    Uninstalling fsspec-2024.5.0:
      Successfully uninstalled fsspec-2024.5.0


In [1]:
from pymilvus import MilvusClient, DataType, Collection
from datasets import load_dataset, load_from_disk

## Preparação do Milvus

In [13]:
# Set up Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

In [14]:
# Create schema
summarizer_schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)

# Add fields to schema
summarizer_schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
summarizer_schema.add_field(field_name="sentence_content", datatype=DataType.VARCHAR, max_length = 1024)
summarizer_schema.add_field(field_name="sentence_vector", datatype=DataType.FLOAT_VECTOR, dim=1536)

# Prepare index parameters
index_params = client.prepare_index_params()

# Add indexes
index_params.add_index(field_name="id")
index_params.add_index(field_name="sentence_vector", index_type="AUTOINDEX", metric_type="COSINE")

# Create a collection
client.create_collection(
    collection_name="news_articles",
    schema=summarizer_schema,
    index_params=index_params
)

## Preparação dos Dados

In [2]:
# Load CNN/DailyMail Dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

In [3]:
train_data = dataset['train']
test_data = dataset['test']

In [4]:
# Content example from dataset
print(train_data[0]['article'])  # Text
print()
print(train_data[0]['highlights'])  # Summary

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [5]:
print(train_data[0])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char